# Plutocracy Report

This is an extensive analysis of the top 60 Decentralised Autonomous Organizations (DAOs), ranked by [treasury size](https://deepdao.io/) to determine the impact of large token holders on their governance.

For this, we create a report based on on-chain data about proposals and voters. This report lists sixty DAOs and assesses their _decisiveness_ of whales_ metric, representing how governance decisions are affected once whale votes are discounted.

In [1]:
# sets up the pynb environment
import os
import sys

from IPython.display import HTML
import pandas as pd

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

Our data sources include DeepDAO and Snapshot, both of which specialize in providing data on DAO governance. We also use Coingecko to find market data for DAO governance tokens.

Using this data, we compile two spreadsheets that act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for up to the last one hundred proposals in each DAO. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters that have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
all_organization_proposals = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_organization_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/plutocracy_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [3]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(space_name), space_proposals)
            for space_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_space_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_organization_proposals)
plutocracy_report_data_filtered = to_organization_map(all_organization_proposals_filtered)


We then ask the following question: **How many whales have voted up to the last 100 proposals for each of the DAOs we analyzed?** Voting power for each of these DAOs varies by the type of [voting strategy](https://docs.snapshot.org/strategies/what-is-a-strategy) outlined in their respective Snapshot spaces.

However, to keep this analysis simple, we assume the common `erc-20-balance` strategy.

In [4]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [5]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "outcome_old",
                "outcome_new"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["whale_vp_proportion","total_vp"], ascending=False)

        try:
            changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]
        except KeyError:
            changed_outcome_proportions[organization] = 0

        space_id = plutocracy_report_data[organization].iloc[0]["proposal_space_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization].index = score_differences_dfs[organization].index.to_series().apply(
            lambda s: f'<a href=http://snapshot.org/#/{space_id}/proposal/{s} rel="noopener noreferrer" target="_blank">{s[0:9]}</a>'
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

In [6]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [7]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
1inch Network,26,604,0.00%
Aavegotchi,47,5624,7.14%
Alchemix,49,2212,4.44%
Ampleforth,6,119,6.25%
Angle Protocol,17,475,10.13%
ApeCoin DAO,192,6812,11.59%
BadgerDAO,251,11169,5.26%
Cryptex,4,24,6.25%
Curve Finance,21,285,20.93%


Here we see that there are quite a few proposals whose outcomes would've changed if whales were not involved, in this analysis, we take a look at some of these cases in Uniswap, Decentraland, Curve Finance and Radicle in particular.

---

Next, we take a look at how governance decisions are affected once whale votes are discounted. We do so by comparing the scores of each proposal and checking whether the outcome of the proposal is changed when whales are filtered out.

We check whether a proposal's outcome changes by checking if the largest vote choice score changes after filtering out whales. Specifically, in python we do the following:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

## Uniswap

### Filtered Proposals Analysis

In [8]:
HTML(score_differences_dfs["Uniswap"].drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
QmP3H8WGL,0.999959,16362720.863718966,False,Add 1bps fee tier (1 tick),Add 1bps fee tier (1 tick)
QmWbgzHJ8,0.999948,19648417.385321181,False,Upgrade to Governor Bravo,Upgrade to Governor Bravo
QmScNLeaj,0.999946,8358752.921668524,False,Upgrade to Governor Bravo,Upgrade to Governor Bravo
QmWe6Krc8,0.999944,7055209.905787433,True,Approve,Deny
0xee48516,0.999941,35237304.209142208,False,Yes,Yes
Qmeza7Lnz,0.999938,23022301.247915581,False,YES,YES
QmZcfEhN6,0.999937,11933325.482144641,False,Yes,Yes
QmUMGd8r2,0.999901,13767227.745737769,False,Yes,Yes
QmNPvJSYq,0.999895,41169566.753146723,False,"Yes, it should be lowered","Yes, it should be lowered"


### Proportion of Outcomes Changed

In [9]:
print(f"{changed_outcome_proportions['Uniswap']} of Uniswap's proposal outcomes change after filtering out whale voting power.")

16.25% of Uniswap's proposal outcomes change after filtering out whale voting power.


### Proposal Analysis

The outcome of the proposal to add a Liquidity Mining Manager (LMM) for the [Optimism-Uniswap Liquidity Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c) changes after removing whales.

If not for whale intervention, "Overnight Finance" would have been chosen as the LMM. Instead, "DeFiEdge" was selected. From the data, we see that without whale intervention, DeFiEdge would've had ~2,708,025 fewer votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [10]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Uniswap"].index.to_series().apply(
    lambda s: "0xfd3d380" in s,
)
proposal_score_differences = score_differences_dfs["Uniswap"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differences,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. LayerZero", the runner-up, received ~37.7% of the total whale voting power. By comparison, "Wormhole" received ~51.1%.

In [11]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Uniswap"].index.to_series().apply(
    lambda s: "0x6b8df36" in s,
)
proposal_score_differences = score_differences_dfs["Uniswap"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differences,2.327546e+07,1.716363e+07,481.545728,3402.436704


## ENS

### Filtered Proposal Outcome Analysis

In [12]:
HTML(score_differences_dfs["ENS"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x41b3509,0.998301,1.670185e+06,False,For,For
0xd7eff78,0.998220,1.670154e+06,False,For,For
0x5788bf0,0.998020,1.671687e+06,False,For,For
0xa245dc7,0.998011,2.282886e+06,False,slobo.eth,slobo.eth
0xa714c25,0.997780,2.299997e+06,False,avsa.eth,avsa.eth
0xdaff050,0.997736,2.325021e+06,False,nick.eth,nick.eth
0x5c96e49,0.994136,2.126140e+06,False,For,For
0x46c7294,0.993740,2.137811e+06,False,For,For
0x4a1aedb,0.993403,2.639657e+06,False,For,For


### Proportion of Outcomes Changed

In [13]:
print(f"{changed_outcome_proportions['ENS']} of ENS' proposal outcomes change after filtering out whale voting power.")

3.57% of ENS' proposal outcomes change after filtering out whale voting power.


## Lido
### Filtered Proposal Analysis

In [14]:
HTML(score_differences_dfs["Lido"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x3b1e5f9,0.999968,5.518937e+07,False,For,For
0x1db8052,0.999959,5.542913e+07,False,For,For
0x1454174,0.999956,6.425261e+07,False,For,For
0x7ac2431,0.999945,5.769020e+07,False,For,For
0xf4beaba,0.999944,6.042543e+07,False,For,For
0x629b547,0.999936,5.200775e+07,False,Support the upgrade plan,Support the upgrade plan
0x8bbd48f,0.999936,3.686686e+07,False,For,For
0x32f6f09,0.999934,5.717509e+07,False,For,For
0x4208b8d,0.999933,4.558269e+07,False,For,For


### Proportion of Outcomes Changed

In [15]:
print(f"{changed_outcome_proportions['Lido']} of Lido's proposal outcomes change after filtering out whale voting power.")

6.00% of Lido's proposal outcomes change after filtering out whale voting power.


[This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed on 16/12/22, onboarded 4 new oracle node operators, and set the quorum to 5/9 identical oracle reports to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11, would've passed.

In [16]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Lido"].index.to_series().apply(
    lambda s: "0xcbf5343" in s,
)
proposal_score_differences = score_differences_dfs["Lido"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differences,92934.922644,5.272312e+07,4535.772055


Here we see whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal and so removing it would make a huge impression on the outcome.

Voting power allocated to this proposal would be low without whale intervention: (only ~9277.7397 LDO would be allocated to this proposal). However, the community would still vote for adding more oracles that [broadcasts the chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0.

## Frax
### Filtered Proposal Outcome Analysis

In [17]:
HTML(score_differences_dfs["Frax"].astype({"total_vp": "float64"}, copy=False).sort_values("whale_vp_proportion", ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x9547dee,0.999550,2.659437e+07,False,For,For
0x6f45c4a,0.998815,2.634672e+07,True,Against,For
0xc44bf39,0.997379,2.715716e+07,False,For,For
0x5d2e624,0.997305,2.715915e+07,False,For,For
0x4540ed6,0.996720,2.717503e+07,False,For,For
0x97d007a,0.992491,2.676523e+07,False,For,For
0x6fc17d9,0.991518,3.633629e+07,False,For,For
0x1127ca8,0.985974,3.375063e+07,False,For,For
0x1c844e1,0.984243,3.377246e+07,False,For,For


### Proportion of Outcomes Changed

In [18]:
print(f"{changed_outcome_proportions['Frax']} of Frax's proposal outcomes change after filtering out whale voting power.")

4.00% of Frax's proposal outcomes change after filtering out whale voting power.


## Decentraland
Let's now look at Decentraland, where the largest holders regularly participate in governance and dominate voting.

### Filtered Proposals Analysis

In [19]:
HTML(score_differences_dfs["Decentraland"].astype({"total_vp": "float64"}, copy=False).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).drop("score_differences", axis=1).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x8b84875,0.999486,4.883961e+06,True,no you can forget about it matey,yes I want someone to pick this up down the line
0x7a6a819,0.999382,5.481279e+06,False,yes,yes
0x187559b,0.999296,5.161839e+06,False,yes,yes
0x8668746,0.999199,5.172165e+06,True,yes,no
0x4913bba,0.998996,7.775241e+06,False,This is a good idea,This is a good idea
0x3262d5a,0.998879,4.874952e+06,False,no,no
0xdfc9d87,0.998616,1.130732e+06,True,no,yes
0x768ee01,0.998182,5.183137e+06,True,yes,no
0xd28d40b,0.998069,5.142210e+06,False,No,No


### Proportion of Outcomes Changed

In [20]:
print(f"{changed_outcome_proportions['Decentraland']} of Decentraland's proposal outcomes change after filtering out whale voting power.")

27.00% of Decentraland's proposal outcomes change after filtering out whale voting power.


For example, [this proposal](https://snapshot.org/#/snapshot.dcl.eth/proposal/0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5) to set a duration period for the tenure of Decentraland DAO committee members saw 99.9% of voting power attributed to whales, with 94.85% of proposal voting power allocated to voting for the proposal not to pass.

In [21]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Decentraland"].index.to_series().apply(
    lambda s: "0x7f6fed8" in s,
)
proposal_score_differences = score_differences_dfs["Decentraland"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Yes,No,Invalid question/options
Scores,168338.243827,1.394610e+06,0.025206
Score Differences,166209.638630,1.392896e+06,0.000000


This is clearly a case of large holders voting to support their own interests. Once whales are filtered out of the votes (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power by comparison.

## Curve Finance

### Filtered Proposals Analysis

In [22]:
HTML(score_differences_dfs["Curve Finance"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xa11d4d1,0.998352,2.609730e+07,False,Yes,Yes
0xa4fb747,0.997837,7.183114e+03,False,当然,当然
0xad241d0,0.995149,1.352219e+05,True,>80%,<10%
0xdaae7dd,0.993704,2.424709e+03,False,Oui,Oui
0xc943136,0.992488,2.089668e+03,False,看好,看好
0x73c09f5,0.992488,2.089666e+03,False,可以,可以
0xd946d40,0.992450,2.056291e+03,False,会,会
0xf18e081,0.992417,2.027366e+03,True,不会,会
0x745856d,0.992091,2.131107e+04,False,"Sure, I am diamond hand","Sure, I am diamond hand"


### Proportion of Outcomes Changed

In [23]:
print(f"{changed_outcome_proportions['Curve Finance']} of Curve Finance's proposal outcomes change after filtering out whale voting power.")

20.93% of Curve Finance's proposal outcomes change after filtering out whale voting power.


One proposal which would've passed if not for whale intervention was this proposal to [add the XSTUSD-3CRV pair](https://snapshot.org/#/curve.eth/proposal/0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5) to Curve's [gauge](https://resources.curve.fi/reward-gauges/understanding-gauges) [controller](https://curve.readthedocs.io/dao-gauges.html#the-gauge-controller) to accrue CRV for liquidity providers of XSTUSD-3CRV.

XSTUSD is a stablecoin deployed on Polkadot and Kusama that's backed by a synthetic token called XOR ([Sora](https://sora.org/)'s native token). The [governance discussion](https://gov.curve.fi/t/proposal-to-add-xstusd-3crv-to-the-gauge-controller/2998/15) about the vote was particularly interesting.

![](./res/curve_governance_shenanigans.png)

What stands out is XSTUSD's comparison with LUNA/UST. This proposal was created before the LUNA/UST [depegging disaster](https://rekt.news/luna-rekt/), but even before that, [quite a](https://twitter.com/runekek/status/1478166276979793922) [few people](https://twitter.com/FreddieRaynolds/status/1463960623402913797) had their concerns about it. So I checked out the first 16 accounts which showed really strong support for this proposal, and almost [every single](https://gov.curve.fi/u/meowtopia) [one was](https://gov.curve.fi/u/LiquidityKing) [created within](https://gov.curve.fi/u/Ryandotrrr) 2 days of the proposal's forum post. Clear signs of governance forum shenanigans, executed to raise hype for a proposal.

In [24]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Curve Finance"].index.to_series().apply(
    lambda s: "0x0eb23ea" in s,
)
proposal_score_differences = score_differences_dfs["Curve Finance"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differences,23462.994316,511330.472493


Just over 4% of voting power for this proposal was allocated by whales to vote "Yes" (which is just over half the total voting power allocated to the "Yes" choice for this proposal), whereas ~89% of whale voting power was allocated to voting "NO" (~97% of total voting power for this choice).

It's important to highlight that whales are also sensible and not always "evil", which I would classify as entities that promote proposals that are detrimental to the DAO for their own interests. It's a good thing that CRV whales didn't have an incentive to pass this proposal as well.

## Radicle
### Filtered Proposals Analysis

In [25]:
HTML(score_differences_dfs["Radicle"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x3ff8024,0.999397,4.301595e+05,False,Yes,Yes
QmVypVPUP,0.927434,1.962803e+06,False,For,For
QmbpshrWc,0.751147,1.291031e+06,False,Unpause swapping,Unpause swapping
0xe9158c0,0.632033,7.565920e+05,False,Yes,Yes
0x704025e,0.624782,4.946306e+06,False,For,For
0x16400b2,0.621619,4.971479e+06,False,For,For
0x646e927,0.605303,6.163640e+06,False,Yes,Yes
QmepPgXwo,0.551155,2.012627e+05,True,Nay,Aye
QmW64iqHQ,0.539397,8.865300e+05,False,For,For


### Proportion of Outcomes Changed

In [26]:
print(f"{changed_outcome_proportions['Radicle']} of Radicle's proposal outcomes change after filtering out whale voting power.")

16.67% of Radicle's proposal outcomes change after filtering out whale voting power.


One such proposal was the one to [distribute RAD remaining](https://snapshot.org/#/gov.radicle.eth/proposal/QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S) following a Liquidity Bootsrapping (LBP) round conducted in February '21. This leftover RAD was proposed to be redistributed to participants of the LBP, i.e. people who bought RAD in this period from the Balancer LBP for RAD tokens.

In [27]:
propsal_choices = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_choices']
mask = score_differences_dfs["Radicle"].index.to_series().apply(
    lambda s: "QmepPgXwo" in s,
)
proposal_score_differences = score_differences_dfs["Radicle"].loc[mask]["score_differences"][0]
proposal_scores = plutocracy_report_data['Radicle'][plutocracy_report_data['Radicle']['proposal_id'] == 'QmepPgXwo5q9GipZFKa32rnxaYoo3LrfRqduinftbU3L3S'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_differences)},
    index=["Scores", "Score Differences"],
)

,Aye,Nay
Scores,78136.278442,123126.401817
Score Differences,20708.270722,90218.650933


Just over 10% of whale voting power for this proposal was allocated to voting "Aye" on this proposal (~27% of voting power allocated to the "Aye" choice came from whales). Whereas ~45% of voting power from whales was allocated to the "Nay" choice (~73% of whale voting power allocated to "Nay").

## Gitcoin
### Filtered Proposals Analysis

In [28]:
HTML(score_differences_dfs["Gitcoin"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x44711d0,0.999678,9.749282e+06,False,YES,YES
0x8a9aecd,0.999631,9.143112e+06,False,YES,YES
0xc6655f1,0.999620,1.196768e+07,False,Yes - Proceed with TD,Yes - Proceed with TD
0x643408b,0.999597,9.300537e+06,False,YES,YES
0xc32ebb4,0.999539,8.099869e+06,False,YES,YES
0x43aa826,0.999534,1.121580e+07,False,YES,YES
0x63e139b,0.999432,9.095121e+06,False,Yes,Yes
0x08bc5d6,0.999431,1.270002e+07,False,Yes,Yes
0x35010ae,0.999387,1.054562e+07,False,"Yes, fund MMM Season 16","Yes, fund MMM Season 16"


### Proportion of Outcomes Changed

In [29]:
print(f"{changed_outcome_proportions['Gitcoin']} of Gitcoin's proposal outcomes change after filtering out whale voting power.")

12.50% of Gitcoin's proposal outcomes change after filtering out whale voting power.


## Euler
### Filtered Proposals Analysis

In [30]:
HTML(score_differences_dfs["Euler"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x93a1390,0.994556,4.152027e+05,False,YES,YES
0x0991c38,0.993948,4.270865e+05,False,"Yes, by 3PM EST 25th Nov 22","Yes, by 3PM EST 25th Nov 22"
0x432e4d3,0.987322,4.008211e+05,False,Claims and self-delegate $SAFE,Claims and self-delegate $SAFE
0x9da517e,0.979825,3.709000e+05,False,Implement this change,Implement this change
0xdbd34a5,0.979729,1.085214e+05,False,YES,YES
0x40874e4,0.964052,5.743725e+05,False,YES -Implement the changes,YES -Implement the changes
0x5b72a41,0.963143,5.740418e+05,False,Yes,Yes
0xe91b62c,0.961527,3.840210e+05,False,NO,NO
0x39151cd,0.949969,6.689393e+05,False,Yes,Yes


### Proportion of Outcomes Changed

In [31]:
print(f"{changed_outcome_proportions['Euler']} of Euler's proposal outcomes change after filtering out whale voting power.")

15.09% of Euler's proposal outcomes change after filtering out whale voting power.


## Gearbox
### Filtered Proposals Analysis

In [32]:
HTML(score_differences_dfs["Gearbox"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xa24c991,0.911661,2.266455e+08,False,"Yes, approve","Yes, approve"
0xc7d4eb1,0.898677,2.119328e+08,False,"Yes, approve","Yes, approve"
0x7155b39,0.891847,2.120120e+08,False,"Yes, approve","Yes, approve"
0x3cbd502,0.888188,2.444278e+08,False,"Yes, approve","Yes, approve"
0x2d77246,0.884413,2.471702e+08,False,Approve,Approve
0x5e348d3,0.876182,2.545753e+08,False,"Yes, approve","Yes, approve"
0x3a4f32b,0.876123,2.444973e+08,False,"Yes, approve","Yes, approve"
0x3719083,0.872143,2.331051e+08,False,Approve OBRA,Approve OBRA
0x23b65a4,0.845524,4.751992e+07,False,"Yes, approve","Yes, approve"


### Proportion of Outcomes Changed

In [33]:
print(f"{changed_outcome_proportions['Gearbox']} of Gearbox's proposal outcomes change after filtering out whale voting power.")

0.00% of Gearbox's proposal outcomes change after filtering out whale voting power.


## SuperRare
### Filtered Proposals Analysis

In [34]:
HTML(score_differences_dfs["SuperRare DAO"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xf84ed27,0.580322,6.061394e+06,False,Yes,Yes
0xd84b09a,0.552082,6.329435e+06,False,Yes,Yes
0x40b7d0e,0.531639,1.639030e+06,False,Yes,Yes
0xc9fa170,0.529507,1.650444e+07,False,Yes,Yes
0x63b7fbb,0.507906,6.957953e+06,False,Yes,Yes
0x1d64af4,0.501453,7.271877e+06,False,Yes,Yes
0x5a51391,0.488250,7.337768e+06,False,Yes,Yes
0xa73cb65,0.470866,6.704130e+06,False,Yes,Yes
0x8d3c065,0.466561,1.313154e+07,False,Yes,Yes


### Proportion of Outcomes Changed

In [35]:
print(f"{changed_outcome_proportions['SuperRare DAO']} of SuperRare DAO's proposal outcomes change after filtering out whale voting power.")

0.00% of SuperRare DAO's proposal outcomes change after filtering out whale voting power.


## Hop
### Filtered Proposals Analysis

In [36]:
HTML(score_differences_dfs["Hop"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xdf8dbd4,0.990016,1.319066e+07,False,For,For
0x8131ad6,0.988627,1.214940e+07,False,Yes,Yes
0x6b51364,0.986493,1.355622e+07,False,Yes,Yes
0x77dfa74,0.986203,1.356527e+07,False,Yes,Yes
0xf1bf932,0.986168,1.356350e+07,False,Yes,Yes
0x6873b06,0.986155,1.097978e+07,False,For,For
0x29c0205,0.984798,1.198519e+07,False,Yes,Yes
0x15937d8,0.984153,1.056193e+07,False,Yes,Yes
0xaa5d0ac,0.982945,9.891047e+06,False,For,For


### Proportion of Outcomes Changed

In [37]:
print(f"{changed_outcome_proportions['Hop']} of Hop's proposal outcomes change after filtering out whale voting power.")

4.17% of Hop's proposal outcomes change after filtering out whale voting power.


## JPEG’d
### Filtered Proposals Analysis

In [38]:
HTML(score_differences_dfs["JPEG’d"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x6f20051,0.980853,4.262598e+07,False,yes,yes
QmU5Ku31d,0.902504,3.055583e+09,False,"Yes, commission the NFTs","Yes, commission the NFTs"
QmZhiDLFX,0.833699,2.277809e+09,False,"Yes, launch NFT auctions","Yes, launch NFT auctions"
0x2f14983,0.815436,8.065378e+08,True,no,yes
0x04ec684,0.804727,8.626524e+07,False,yes,yes
0x2c3a786,0.753776,3.241814e+09,False,Diversify,Diversify
QmahuLsSu,0.751152,3.803228e+09,False,Design 3,Design 3
0x8b2f168,0.749019,3.238020e+08,True,"Yes, launch the index",Abstain
0x55723c4,0.742123,2.542658e+09,False,Yes approve new JPEG boost,Yes approve new JPEG boost


### Proportion of Outcomes Changed

In [39]:
print(f"{changed_outcome_proportions['JPEG’d']} of JPEG’d's proposal outcomes change after filtering out whale voting power.")

7.27% of JPEG’d's proposal outcomes change after filtering out whale voting power.


## Merit Circle
### Filtered Proposals Analysis

In [40]:
HTML(score_differences_dfs["Merit Circle"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xd155513,0.701893,4.135070e+06,False,No,No
QmQkG8aWr,0.693401,5.812888e+06,False,Yes,Yes
0x0915881,0.664194,4.153706e+06,False,No,No
0xb304e09,0.625963,1.184047e+07,False,Yes,Yes
0xc5d25c7,0.619293,3.233531e+06,False,MC/ETH,MC/ETH
0x1466420,0.611997,6.576268e+06,False,Yes,Yes
0xf06600b,0.609372,5.918585e+06,False,Yes,Yes
0xbd4b35e,0.597107,8.199661e+06,False,MC/USDC,MC/USDC
0x732345c,0.594546,6.370837e+06,False,Yes,Yes


### Proportion of Outcomes Changed

In [41]:
print(f"{changed_outcome_proportions['Merit Circle']} of Merit Circle's proposal outcomes change after filtering out whale voting power.")

0.00% of Merit Circle's proposal outcomes change after filtering out whale voting power.


## Aavegotchi
### Filtered Proposals Analysis

In [42]:
HTML(score_differences_dfs["Aavegotchi"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x1dcc158,0.845668,1.086557e+07,False,Fund Completed Work/Bounties,Fund Completed Work/Bounties
0x9e71af4,0.829492,7.556619e+06,False,Yes. Implement Strategy,Yes. Implement Strategy
0x1ea93e1,0.821143,1.052393e+07,False,A,A
0x2ab8f9e,0.812639,1.474443e+07,False,"Yes, agree to 25/25/25/25 split","Yes, agree to 25/25/25/25 split"
0x2d9a7ec,0.810330,1.459242e+07,False,Yes. Form and fund the CTT.,Yes. Form and fund the CTT.
0xe95a5ed,0.809063,1.267284e+07,False,Keep spillover turned off,Keep spillover turned off
0x0a5979a,0.805901,1.346150e+07,True,Yes. Launch Round 2 parcel roll,No. Wait until the great battle.
0x6fd66b3,0.804271,1.378177e+07,False,YES Fund 4.5k ghst partnership,YES Fund 4.5k ghst partnership
0x0f8ae0b,0.803015,1.604231e+07,False,"Yes, turn off the GHST Curve","Yes, turn off the GHST Curve"


### Proportion of Outcomes Changed

In [43]:
print(f"{changed_outcome_proportions['Aavegotchi']} of Aavegotchi's proposal outcomes change after filtering out whale voting power.")

7.14% of Aavegotchi's proposal outcomes change after filtering out whale voting power.


## Rook
### Filtered Proposals Analysis

In [44]:
HTML(score_differences_dfs["Rook"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0xa95d98a,0.823478,19612.908172,False,No objection,No objection
0x87e9189,0.787806,15575.916427,False,No objection,No objection
0x74bf937,0.756495,16243.977807,False,No objection,No objection
0x0b3ad00,0.734236,16736.431371,False,No objection,No objection
0xcf296f7,0.733281,22067.657107,False,No objection,No objection
0x44e5b1c,0.700316,19669.863695,False,No objection,No objection
0x68f5e4c,0.684806,20115.343064,False,No objection,No objection
0x5715c13,0.650868,18414.550127,False,No objection,No objection
0x5ab5397,0.641678,19590.509500,False,No objection,No objection


### Proportion of Outcomes Changed

In [45]:
print(f"{changed_outcome_proportions['Rook']} of Rook's proposal outcomes change after filtering out whale voting power.")

5.41% of Rook's proposal outcomes change after filtering out whale voting power.


## ApeCoin DAO
### Filtered Proposals Analysis

In [46]:
HTML(score_differences_dfs["ApeCoin DAO"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x7e43228,0.990179,6.256527e+05,False,I confirm this is a test vote.,I confirm this is a test vote.
0x8e360c0,0.981936,4.379614e+06,False,In Favor,In Favor
0xfe0e75a,0.980226,1.659834e+06,False,Against,Against
0x5840047,0.973400,1.550751e+06,False,In favor,In favor
0x564cd76,0.971424,1.286369e+06,False,Against,Against
0x6ed3c09,0.971044,2.076817e+06,False,In favor,In favor
0xf9d857c,0.968874,2.008295e+06,True,Against,In favor
0x5d449bd,0.967873,1.009663e+06,False,In favor,In favor
0xbff08fe,0.965701,4.219239e+05,False,In favor,In favor


### Proportion of Outcomes Changed

In [47]:
print(f"{changed_outcome_proportions['ApeCoin DAO']} of ApeCoin DAO's proposal outcomes change after filtering out whale voting power.")

11.59% of ApeCoin DAO's proposal outcomes change after filtering out whale voting power.


## Klima DAO
### Filtered Proposals Analysis

In [48]:
HTML(score_differences_dfs["Klima DAO"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x930b600,0.996920,504433.644339,False,For,For
0xa2d8bd4,0.996727,474787.421434,False,For: split fee 80/20,For: split fee 80/20
0xa50ab29,0.987467,425434.738707,False,For,For
0x8e4ef40,0.980098,750178.179232,False,For,For
QmZUSGcMq,0.840598,9264.591987,False,48 hours,48 hours
0xd358805,0.727192,195233.769979,False,For: Build NCT liquidity,For: Build NCT liquidity
0xe69c1cc,0.699951,335036.922800,False,For: Reduce AKR to 20%,For: Reduce AKR to 20%
0x23ad839,0.680467,52686.991299,False,For: Reduce rate to match actual,For: Reduce rate to match actual
0xe9de3ba,0.677817,236863.094312,False,For: Bolster UBO & NBO liquidity,For: Bolster UBO & NBO liquidity


### Proportion of Outcomes Changed

In [49]:
print(f"{changed_outcome_proportions['Klima DAO']} of Klima DAO's proposal outcomes change after filtering out whale voting power.")

0.00% of Klima DAO's proposal outcomes change after filtering out whale voting power.


## Hector Network
### Filtered Proposals Analysis

In [50]:
HTML(score_differences_dfs["Hector Network"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x55918aa,0.740052,100942.001096,True,No,Yes
0xf61dfa6,0.735902,111331.738215,False,Yes,Yes
0x37de6bd,0.731288,121959.039539,False,Illuvium,Illuvium
0x74b62bc,0.728766,107968.121715,False,Yes,Yes
0x6ec98d7,0.716335,77377.042178,False,Yes,Yes
0xc4e6334,0.714887,125301.767306,False,Yes,Yes
0x4475e8a,0.712459,131082.049927,False,Yes,Yes
0x205c78d,0.712411,97912.273004,False,Yes,Yes
0x754a389,0.695676,192498.291501,False,NO,NO


### Proportion of Outcomes Changed

In [51]:
print(f"{changed_outcome_proportions['Hector Network']} of Hector Network's proposal outcomes change after filtering out whale voting power.")

5.80% of Hector Network's proposal outcomes change after filtering out whale voting power.


## Vesta
### Filtered Proposals Analysis

In [52]:
HTML(score_differences_dfs["Vesta"].astype({"total_vp": "float64"}, copy=False).drop("score_differences", axis=1).sort_values(["whale_vp_proportion", "total_vp"], ascending=False).to_html(render_links=True, escape=False))

,whale_vp_proportion,total_vp,outcome_changed,outcome_old,outcome_new
Proposal ID,,,,,
0x4a478d3,0.999585,5.153223e+05,False,Restructure as outlined,Restructure as outlined
0x02ada85,0.987708,3.364918e+05,False,add VRR and halt redemption,add VRR and halt redemption
0xd22f9ab,0.902557,1.218759e+06,True,agree with this term and proceed,re-evaluate the term
0x128bc73,0.817738,2.568059e+05,False,For Unvested Token Swap,For Unvested Token Swap
0xe36d551,0.810129,1.234371e+06,False,"Yes, engage Risk DAO","Yes, engage Risk DAO"
0xec88562,0.663747,3.886962e+04,False,For,For
0x5887b4d,0.281922,4.341528e+05,False,Remove GMX performance fee,Remove GMX performance fee
0x938268e,0.078807,2.727852e+05,False,For,For


### Filtered Proposals Analysis

In [53]:
print(f"{changed_outcome_proportions['Vesta']} of Vesta's proposal outcomes change after filtering out whale voting power.")

12.50% of Vesta's proposal outcomes change after filtering out whale voting power.
